In [ ]:
# @title Run to download the required libraries and models
print("Downloading and installing libraries...")
%pip install ultralytics supervision roboflow ipywidgets > /dev/null 2>&1
print("Done")
print("Downloading the model...")
!pip install gdown > /dev/null 2>&1
import gdown
file_id = '12q1xnr7_Enz_BwfP8G0DtAM4Z_i2KYhi'
destination = 'yolo11.pt'
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=True)
print("Done")
print("Setup complete!")


In [ ]:
# @title Run the model (use T4 GPU for best resutls)
import ipywidgets as widgets
from IPython.display import display
from ultralytics import YOLO
import os
from google.colab import files
import torch

def predict(source_path):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO('yolo11.pt').to(device)
    results = model.predict(source=source_path, conf=0.25, save=True)
    output_path = results[0].save_dir
    image_name = os.listdir(output_path)[0]
    image_path = os.path.join(output_path, image_name)
    final_image_path = f"/content/{image_name}"
    os.rename(image_path, final_image_path)
    return final_image_path

uploaded = files.upload()

for filename in uploaded.keys():
    source_input = widgets.Text(value=filename, description='Source File:', placeholder='Enter image filename')

button = widgets.Button(description="Run YOLO")

def on_button_click(b):
    output_image_path = predict(source_input.value)
    files.download(output_image_path)

button.on_click(on_button_click)

display(source_input)
display(button)
